In [12]:
var d3 = require('d3');
var pmongo = require('promised-mongo');
var fs = require('fs');
var csvWriter = require('csv-write-stream');

undefined

In [2]:
var dbUrl = 'mongodb://localhost/tateGalleryMeta';
var db = pmongo(dbUrl);
var artColl = db.collection('artworks');

undefined

In [11]:
var turner;
artColl.find({all_artists: "Joseph Mallord William Turner"}).toArray()
    .then(docs => { 
        turner = docs;
    })
    .then(() => {
        console.log('done');
        turner.sort((a, b) => {
            return a.dateRange.startYear - b.dateRange.startYear
        })});

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

done


In [12]:
turner[100]

{ _id: 56e71f7e759a662e391cb1a5,
  acno: 'A01009',
  acquisitionYear: 1925,
  all_artists: 'Joseph Mallord William Turner',
  catalogueGroup: 
   { accessionRanges: 'A00911-A01015; A01112-A01159; A01188',
     completeStatus: 'COMPLETE',
     groupType: null,
     id: 65249,
     shortTitle: 'Etchings and Engravings for the \'Liber Studiorum\'' },
  classification: 'on paper, print',
  contributorCount: 1,
  contributors: 
   [ { birthYear: 1775,
       date: '1775–1851',
       displayOrder: 1,
       fc: 'Joseph Mallord William Turner',
       gender: 'Male',
       id: 558,
       mda: 'Turner, Joseph Mallord William',
       role: 'artist',
       startLetter: 'T' } ],
  creditLine: 'Presented by A. Acland Allen through the Art Fund 1925',
  dateRange: { endYear: 1812, startYear: 1812, text: '1812' },
  dateText: '1812',
  depth: '',
  dimensions: 'image: 178 x 261 mm',
  foreignTitle: null,
  groupTitle: 'Etchings and Engravings for the \'Liber Studiorum\'',
  height: '261',
  id:

For ease of use, save the turner stuff to a new collection 'turnerColl'
pipline should be `d` but not sure why
it doesn't seem to work here...

In [11]:

var tWritStream = fs.createWriteStream('turnerAll.json');

undefined

In [15]:
var turnerColl = db.collection('turnerColl');

undefined

do a projection on the data and limit some fields...

In [61]:
var turnerSelected;
turnerColl.find({}, {
    "_id": 0,
    "classification": 1, "dateRange.startYear": 1, "height": 1, "width": 1,
    "medium":1,"thumbnailUrl":1 ,
    "subjectCount": 1, "subjects": 1,
    "title": 1,"units":1, "url":1}).toArray()
    .then(docs => { 
        turnerSelected = docs;
    })
    .then(() => {
        console.log('done');
    console.log(turnerSelected[0]);
});

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

done
{ classification: 'painting',
  dateRange: { startYear: 1837 },
  height: '1111',
  medium: 'Oil paint on canvas',
  subjectCount: 10,
  subjects: 
   { children: [ [Object], [Object], [Object], [Object] ],
     id: 1,
     name: 'subject' },
  thumbnailUrl: 'http://www.tate.org.uk/art/images/work/A/A00/A00909_8.jpg',
  title: 'The Death of Actaeon, with a Distant View of Montjovet, Val d’Aosta',
  units: 'mm',
  url: 'http://www.tate.org.uk/art/artworks/turner-the-death-of-actaeon-with-a-distant-view-of-montjovet-val-daosta-a00909',
  width: '1492' }


In [26]:
tWritStream.on('finish', () => {console.log('done')});

WriteStream {
  _writableState: 
   WritableState {
     objectMode: false,
     highWaterMark: 16384,
     needDrain: false,
     ending: false,
     ended: false,
     finished: false,
     decodeStrings: true,
     defaultEncoding: 'utf8',
     length: 0,
     writing: false,
     corked: 0,
     sync: true,
     bufferProcessing: false,
     onwrite: [Function],
     writecb: null,
     writelen: 0,
     bufferedRequest: null,
     lastBufferedRequest: null,
     pendingcb: 0,
     prefinished: false,
     errorEmitted: false },
  writable: true,
  domain: null,
  _events: { finish: [ [Object], [Function] ] },
  _eventsCount: 1,
  _maxListeners: undefined,
  path: 'turnerAll.json',
  fd: 33,
  flags: 'w',
  mode: 438,
  start: undefined,
  pos: undefined,
  bytesWritten: 0 }

And now let's try to pipe the db query to output json

In [27]:
var JSONStream = require('JSONStream');
turnerColl.find({}, {
    "_id": 0,
    "classification": 1, "dateRange.startYear": 1, "height": 1, "width": 1,
    "medium":1,"thumbnailUrl":1 ,
    "subjectCount": 1, "subjects": 1,
    "title": 1,"units":1, "url":1}).pipe(JSONStream.stringify()).pipe(tWritStream);

WriteStream {
  _writableState: 
   WritableState {
     objectMode: false,
     highWaterMark: 16384,
     needDrain: false,
     ending: false,
     ended: false,
     finished: false,
     decodeStrings: true,
     defaultEncoding: 'utf8',
     length: 0,
     writing: false,
     corked: 0,
     sync: true,
     bufferProcessing: false,
     onwrite: [Function],
     writecb: null,
     writelen: 0,
     bufferedRequest: null,
     lastBufferedRequest: null,
     pendingcb: 0,
     prefinished: false,
     errorEmitted: false },
  writable: true,
  domain: null,
  _events: 
   { finish: [ [Object], [Function] ],
     drain: [Function: ondrain],
     error: [Function: onerror],
     close: [Function: cleanup] },
  _eventsCount: 4,
  _maxListeners: undefined,
  path: 'turnerAll.json',
  fd: 33,
  flags: 'w',
  mode: 438,
  start: undefined,
  pos: undefined,
  bytesWritten: 0 }

done


Let's see whether the Turner works has any unusal classifications:

In [30]:
turnerColl.distinct('classification').then((data) => {console.log(data)})

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

[ 'painting',
  'on paper, unique',
  'on paper, print',
  'block for printing' ]


I guess not...

In [33]:
turnerColl.distinct('medium').then((data) => {console.log(data, data.length)})

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

[ 'Oil paint on canvas',
  'Mixed media and graphite on paper',
  'Etching and engraving on paper',
  'Etching on paper',
  'Etching and mezzotint on paper',
  'Etching, mezzotint and drypoint on paper',
  null,
  'Etching, aquatint and mezzotint on paper',
  'Etching, drypoint and mezzotint on paper',
  'Etching and engraving on paper\r\n\r\n',
  'Etching and mezzotint engraving on paper',
  'Print on paper',
  'Etching and aquatint on paper',
  'Etching, mezzotint and aquatint on paper',
  'Mezzotint on paper',
  'Photograph on paper',
  'Mezzotint engraving on paper',
  'Pen and ink and watercolour on paper',
  'Gouache, graphite and watercolour on paper',
  'Graphite and pen on paper',
  'Graphite and watercolour on paper',
  'Graphite on paper',
  'Watercolour on paper',
  'Ink wash, pen and ink, graphite and watercolour on paper',
  'Graphite, watercolour and pen and ink on paper',
  'Pen and ink on paper',
  'Pen and ink and graphite on paper',
  'Chalk, gouache and watercolour 

On the other hand, 'medium' returns 187 distinct groups, but tbh some of them can probably be combined, e.g. Oil paint on x...

In [43]:
var categories = [];
console.log(categories)
turnerColl.distinct('medium').then((docs) => {
    console.log(docs[0]);
    docs.forEach(doc => {
        if (doc == null) {return}
        var type = doc.split(' on')[0];
        if (categories.indexOf(type) < 0) {
            categories.push(type);
        }
    });
    console.log('done',categories, categories.length)
})

[]


Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

Oil paint on canvas
done [ 'Oil paint',
  'Mixed media and graphite',
  'Etching and engraving',
  'Etching',
  'Etching and mezzotint',
  'Etching, mezzotint and drypoint',
  'Etching, aquatint and mezzotint',
  'Etching, drypoint and mezzotint',
  'Etching and mezzotint engraving',
  'Print',
  'Etching and aquatint',
  'Etching, mezzotint and aquatint',
  'Mezzotint',
  'Photograph',
  'Mezzotint engraving',
  'Pen and ink and watercolour',
  'Gouache, graphite and watercolour',
  'Graphite and pen',
  'Graphite and watercolour',
  'Graphite',
  'Watercolour',
  'Ink wash, pen and ink, graphite and watercolour',
  'Graphite, watercolour and pen and ink',
  'Pen and ink',
  'Pen and ink and graphite',
  'Chalk, gouache and watercolour',
  'Chalk and gouache',
  'Chalk',
  'Chalk and watercolour',
  'Chalk, gouache and graphite',
  'Graphite and ink',
  'Graphite and pen and ink',
  'Pen and watercolour and pen and ink',
  'Pen and graphite',
  'Pen and ink, graphite and watercolour',

A bit better but still lots of groups... Let's try with just the first word (not the most intelligent, I know)

In [45]:
var categories2 = [];
turnerColl.distinct('medium').then((docs) => {
    console.log(docs[0]);
    docs.forEach(doc => {
        if (doc == null) {return}
        var type = doc.split(' ')[0];
        //also remove e.g. ',' after words to make sure e.g. 'chalk' and 'chalk,'
        //being classified as distinct
        type = type.replace(/[^\w\s]|_/g, "");
        if (categories2.indexOf(type) < 0) {
            categories2.push(type);
        }
    });
    console.log('done',categories2, categories2.length)
})

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

Oil paint on canvas
done [ 'Oil',
  'Mixed',
  'Etching',
  'Print',
  'Mezzotint',
  'Photograph',
  'Pen',
  'Gouache',
  'Graphite',
  'Watercolour',
  'Ink',
  'Chalk',
  'Crayon',
  'Charcoal',
  'Engraving',
  'Tracing',
  'Wax',
  'Bodycolour',
  'Line',
  'Marbled',
  'Offset',
  'On',
  'Copper' ] 23


On the other hand, it would be useful to get the first level of the 'subjects' field, and to turn the 'dateRange' into just the startYear...

In [64]:
turnerSelected.forEach(item => {
    var subjects = [];
    if (!item.subjects || !item.subjects.children){
        item.subjectArr = [];
    }else {
        item.subjects.children.forEach(d => {
            subjects.push(d.name)
        });
        item.subjectArr = subjects;
    }
    
    if(!item.dateRange){
        item.year = null;
    } else {
        item.year = item.dateRange.startYear;
    }
    
});
console.log(turnerSelected[10])

{ classification: 'on paper, print',
  dateRange: { startYear: 1807 },
  height: '259',
  medium: 'Etching and mezzotint on paper',
  subjectCount: 15,
  subjects: 
   { children: 
      [ [Object],
        [Object],
        [Object],
        [Object],
        [Object],
        [Object],
        [Object],
        [Object] ],
     id: 1,
     name: 'subject' },
  thumbnailUrl: 'http://www.tate.org.uk/art/images/work/A/A00/A00918_8.jpg',
  title: 'Scene on the French Coast, engraved by Charles Turner',
  units: 'mm',
  url: 'http://www.tate.org.uk/art/artworks/turner-scene-on-the-french-coast-engraved-by-charles-turner-a00918',
  width: '183',
  subjectArr: 
   [ 'places',
     'objects',
     'nature',
     'society',
     'people',
     'architecture',
     'work and occupations',
     'leisure and pastimes' ],
  year: 1807 }


undefined

In [65]:
turnerSelected[100]

{ classification: 'on paper, print',
  dateRange: { startYear: 1812 },
  height: '265',
  medium: 'Etching and mezzotint on paper',
  subjectCount: 17,
  subjects: 
   { children: [ [Object], [Object], [Object], [Object], [Object] ],
     id: 1,
     name: 'subject' },
  thumbnailUrl: 'http://www.tate.org.uk/art/images/work/A/A01/A01007_8.jpg',
  title: 'River Wye, engraved by W. Annis',
  units: 'mm',
  url: 'http://www.tate.org.uk/art/artworks/turner-river-wye-engraved-by-w-annis-a01007',
  width: '184',
  subjectArr: 
   [ 'places',
     'nature',
     'leisure and pastimes',
     'architecture',
     'people' ],
  year: 1812 }

In [59]:
turnerSelected[90]

{ classification: 'on paper, print',
  dateRange: { startYear: 1812 },
  height: '255',
  medium: 'Etching, aquatint and mezzotint on paper',
  subjectCount: 12,
  subjects: [],
  thumbnailUrl: 'http://www.tate.org.uk/art/images/work/A/A00/A00997_8.jpg',
  title: 'Bridge and Goats, engraved by F.C. Lewis',
  units: 'mm',
  url: 'http://www.tate.org.uk/art/artworks/turner-bridge-and-goats-engraved-by-fc-lewis-a00997',
  width: '182',
  year: 1812 }

In [60]:
turnerSelected[900
              ]

{ classification: 'on paper, unique',
  dateRange: { startYear: 1796 },
  height: '129',
  medium: null,
  subjectCount: 0,
  thumbnailUrl: null,
  title: 'Blank',
  units: 'mm',
  url: 'http://www.tate.org.uk/art/artworks/turner-blank-d00771',
  width: '110',
  subjects: [],
  year: 1796 }

Now, lets try to pipe the parsing to a json file...

In [87]:
var parser = function(item){
    var subjects = [];
    if (!item.subjects || !item.subjects.children){
        item.subjectArr = [];
    }else {
        item.subjects.children.forEach(d => {
            subjects.push(d.name)
        });
        item.subjectArr = subjects;
    }
    
    if(!item.dateRange){
        item.year = null;
    } else {
        item.year = item.dateRange.startYear;
    }
    return item;
}

undefined

In [84]:
var tWritStream = fs.createWriteStream('turner2.json');
tWritStream.on('finish', () =>{console.log('done')})

WriteStream {
  _writableState: 
   WritableState {
     objectMode: false,
     highWaterMark: 16384,
     needDrain: false,
     ending: false,
     ended: false,
     finished: false,
     decodeStrings: true,
     defaultEncoding: 'utf8',
     length: 0,
     writing: false,
     corked: 0,
     sync: true,
     bufferProcessing: false,
     onwrite: [Function],
     writecb: null,
     writelen: 0,
     bufferedRequest: null,
     lastBufferedRequest: null,
     pendingcb: 0,
     prefinished: false,
     errorEmitted: false },
  writable: true,
  domain: null,
  _events: { finish: [ [Object], [Function] ] },
  _eventsCount: 1,
  _maxListeners: undefined,
  path: 'turner2.json',
  fd: null,
  flags: 'w',
  mode: 438,
  start: undefined,
  pos: undefined,
  bytesWritten: 0 }

In [85]:

var Transform = require('stream').Transform,
    util = require('util');
 
var TransformStream = function() {
  Transform.call(this, {objectMode: true});
};
util.inherits(TransformStream, Transform);
 
TransformStream.prototype._transform = function(chunk, encoding, callback) {
    parser(chunk);
    callback();
};

var ts = new TransformStream();
turnerColl.find({}, {
    "_id": 0,
    "classification": 1, "dateRange.startYear": 1, "height": 1, "width": 1,
    "medium":1,"thumbnailUrl":1 ,
    "subjectCount": 1, "subjects": 1,
    "title": 1,"units":1, "url":1}).pipe(ts).pipe(JSONStream.stringify()).pipe(tWritStream);

WriteStream {
  _writableState: 
   WritableState {
     objectMode: false,
     highWaterMark: 16384,
     needDrain: false,
     ending: false,
     ended: false,
     finished: false,
     decodeStrings: true,
     defaultEncoding: 'utf8',
     length: 0,
     writing: false,
     corked: 0,
     sync: true,
     bufferProcessing: false,
     onwrite: [Function],
     writecb: null,
     writelen: 0,
     bufferedRequest: null,
     lastBufferedRequest: null,
     pendingcb: 0,
     prefinished: false,
     errorEmitted: false },
  writable: true,
  domain: null,
  _events: 
   { finish: [ [Object], [Function] ],
     drain: [Function: ondrain],
     error: [Function: onerror],
     close: [Function: cleanup] },
  _eventsCount: 4,
  _maxListeners: undefined,
  path: 'turner2.json',
  fd: 40,
  flags: 'w',
  mode: 438,
  start: undefined,
  pos: undefined,
  bytesWritten: 0 }

done


In [91]:
turnerColl.distinct('subjects.children.name').then((docs) => { console.log(done, docs)})

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

In [3]:
var artistColl = db.collection('artists');

undefined

In [5]:
artistColl.distinct('birth', (err, docs) => { console.log('done', docs)})

Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

Getting the distinct places of birth of the artists and saving to csv

In [18]:
artistColl.distinct('birth.place.name').then(doc => {
    console.log(doc.length, doc);
    var writer = csvWriter({headers: ['place']});
    writer.pipe(fs.createWriteStream('artistsBirthplaces.csv'));
    doc.forEach(item => {
         writer.write([item]);
    })
    writer.end();
})


Promise {
  _d: 
   { p: [Circular],
     c: [],
     a: undefined,
     s: 0,
     d: false,
     v: undefined,
     h: false,
     n: false } }

1013 [ 'Polska',
  'Philadelphia, United States',
  'Springfield, United States',
  'Leicestershire, United Kingdom',
  'Wigan, United Kingdom',
  'Tel Aviv-Yafo, Yisra\'el',
  'Kiel, Deutschland',
  'New York, United States',
  'Leeds, United Kingdom',
  'Isleworth, United Kingdom',
  'Kirkcaldy, United Kingdom',
  'Worcester, United Kingdom',
  'London, United Kingdom',
  'Northampton, United Kingdom',
  'Tuszyn, Polska',
  'Reading, United Kingdom',
  'Udine, Italia',
  'Ayacucho, Argentina',
  'Genève, Schweiz',
  'Hämeenlinna, Suomi',
  'Beijing, Zhonghua',
  'Edinburgh, United Kingdom',
  'Carpignano Sesia, Italia',
  'Izmir, Türkiye',
  'Kirkuk, Al-‘Iraq',
  'Berlin, Deutschland',
  'Bottrop, Deutschland',
  'Chicago, United States',
  'Greenwich, United Kingdom',
  'Bruxelles, België',
  'Maidstone, United Kingdom',
  'Moskva, Rossiya',
  'Alloa, United Kingdom',
  'Southampton, United Kingdom',
  'Paris, France',
  'Lambeth, United Kingdom',
  'Worcestershire, United Kingdom',

In [16]:
var doc = ['Philadelphia, United States',
  'Springfield, United States',
  'Leicestershire, United Kingdom',
  'Wigan, United Kingdom']
var writer = csvWriter({headers: ['place']});
    writer.pipe(fs.createWriteStream('artistsBirthplaces.csv'));
    writer.write(['Philadelphia, United States',
  'Springfield, United States',
  'Leicestershire, United Kingdom',
  'Wigan, United Kingdom']);
    writer.end();

undefined